## S&P Global - AUTOMATIC BASELINE CORRECTION
### Misael M. Morales, Carlos Torres-Verdin, and Michael Pyrcz
#### 2024, The University of Texas at Austin
***

In [ ]:
from utils import *

***
# LOG ANALYSIS
***

In [ ]:
spl = SPLogAnalysis()

In [ ]:
# ### WARNING: this takes a long time to run, only do it if you need to
# headers = spl.read_all_headers()

In [ ]:
spl.plot_ccs_sand_wells(figsize=(8,3))
spl.plot_survey(figsize=(10,3))

In [ ]:
well_name = '17700004060000'
spl.plot_well(well_name)
spl.make_arima(well_name)

***
# SP BASELINE CORRECTION
***

In [ ]:
from utils import *

In [ ]:
blc = BaselineCorrection()

In [ ]:
blc.load_logs(preload=True, showfig=False)

In [ ]:
blc.scale_and_random_split(scaler='standard', showfig=False)

In [ ]:
blc.make_model()

In [ ]:
blc.make_predictions()

In [ ]:
# def calc_uq():
#     p5 = data.p5
#     p95 = data.p95
#     entire_uq = p95 - p5


In [ ]:
def calc_csh(well):
    # p5(well)
    # p95(well)
    # csh = (well - well-p5) / (p95 - p5)

    csh = (well - well.min()) / (well.max() - well.min())
    return csh

In [ ]:
nrows, ncols = 3, 8
fig, axs = plt.subplots(nrows, ncols, figsize=(20, 12))
data = y_test_pred

k, mult = 0, 3
for i in range(nrows):
    for j in range(ncols):

        mask = ~np.isnan(X_test[k,:,0])
        index = X_test[k,:,0][mask]
        y  = y_test[k][mask]
        yh = y_test_pred[k][mask]
        csh = calc_csh(yh)

        axs[i,j].plot(y, index, c='darkmagenta', label='True Corr.')
        axs[i,j].plot(yh, index, c='k', label='Pred Corr.')
        ax2 = axs[i,j].twiny()
        ax2.plot(csh, index, c='tab:green', label='$C_{sh}$')

        axs[i,0].set_ylabel('DEPTH [ft]', weight='bold')
        axs[-1,j].set_xlabel('SP [mV]', weight='bold')
        axs[i,j].set_xlim(-200, 50)
        axs[i,j].grid(True, which='both')
        axs[i,j].invert_yaxis()
        k += 1*mult

plt.tight_layout(); plt.show()

***
# END